In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os
import logging
import wandb


In [ ]:
# Set the TensorFlow log level to only display errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Set the logging level to ERROR
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [15]:
from wandb.integration.keras import  WandbCallback

In [8]:
!wandb login 6831aadea925e84318b1e9b4086360f219939fd5

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Avnish\.netrc


In [16]:
# Initialize Wandb
wandb.init(
    project="food_not_food",
    config={
        "learning_rate": 0.001,
        "architecture": "EfficientNetV2B2",
        "dataset": "Food-101 and filtered ImageNet-1k",
        "epochs": 10,
    }
)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: avnishs17 (sukuna). Use `wandb login --relogin` to force relogin


In [27]:
# Load the EfficientNetV2B2 model with pre-trained weights on ImageNet
model = EfficientNetV2B2(weights='imagenet', include_top=False)

In [28]:
model.trainable =False

In [29]:
# Create functional model with additional layers
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

# Construct the functional model
model = tf.keras.Model(inputs, outputs)

In [30]:
# Define directories
train_dir = 'D:/food_not_food/data/train'
test_dir = 'D:/food_not_food/data/test'

In [31]:
# Load datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int'
)

Found 11802 files belonging to 2 classes.


In [32]:
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int'
)

Found 2248 files belonging to 2 classes.


In [33]:
# Preprocess the datasets
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [34]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
# Train the model
num_epochs = 10

history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset, callbacks=[WandbCallback()])


Epoch 1/10
369/369 [==============================] - ETA: 0s - loss: 0.0671 - accuracy: 0.9792

wandb: Adding directory to artifact (d:\food_not_food\wandb\run-20240515_041816-jdvge9v7\files\model-best)... Done. 0.3s


369/369 [==============================] - 1002s 3s/step - loss: 0.0671 - accuracy: 0.9792 - val_loss: 0.0237 - val_accuracy: 0.9938
Epoch 2/10
179/369 [=============>................] - ETA: 6:32 - loss: 0.0308 - accuracy: 0.9888

KeyboardInterrupt: 

In [ ]:
wandb.finish()

In [ ]:
wandb.finish()

In [ ]:
# # Unfreeze some layers in the base model and continue training
# for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers
#     layer.trainable = True

# # Recompile the model with a lower learning rate
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Continue training the model
# history_fine_tune = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)


In [12]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Accuracy: {accuracy:.2f}')

62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 1.0000 - loss: 0.0025
Test Accuracy: 1.00
